In [11]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import pytz
import yfinance as yf

In [12]:
tolerance = 0.
k = 12
h = 2

# rule = 'BME'

time_range = '5y'

start_time = '2019-12-31'
end_time = '2024-01-01'

# Vol scaling
vol_flag = 0                  # Set flag to 1 for vol targeting
if vol_flag == 1:
    target_vol = 0.4 # Can be changed
else:
    target_vol = 'no target vol'


In [13]:
# tickers = pd.read_html( 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

# # stock_list = tickers['Symbol'].to_list()
# print(tickers)
# stock_list = tickers['Symbol'][:50]

In [14]:
def US_Stock_data():

    stock_list = pd.read_html( 'https://en.wikipedia.org/wiki/EURO_STOXX_50')[4]['Ticker'][1:].to_list()
    futures = pd.DataFrame(columns= stock_list)

    time_index = list(yf.Ticker(stock_list[0]).history(period = time_range,start = start_time, end = end_time).index)
    flag = 0

    for symbol in stock_list:
        df = yf.Ticker(symbol).history(period = time_range, start = start_time, end = end_time)
        df = pd.DataFrame(df['Close'])
        i = 0
        daily_return = []
        for k in df['Close']:
            if i != 0:
                daily_return.append(float((k-i)/i))
            else:
                daily_return.append(float(0))
            i = k
        if time_index != list(df.index): flag +=1
        try:
            futures[symbol] = daily_return
        except:
            print(symbol)
            print(len(daily_return)) # Check differnet time index
            while len(daily_return) < len(futures):
                daily_return.insert(0,float(0))
            futures[symbol] = daily_return

    futures.index = time_index
    futures = futures.iloc[1:, :]


    # print(flag) # Check differnet time index

    futures['Date'] = pd.to_datetime(futures.index, format='%Y-%m-%d')
    futures.set_index('Date', inplace=True)

    return futures

In [15]:
def Volatility_scale(data):

    std_index = data.index

    daily_index = pd.DataFrame(index=std_index)

    for oo in data.columns:
        returns = data[oo]
        returns.dropna(inplace=True)
    
        # first_date = returns.index[0].strftime("%Y-%m-%d")   # store this to show when data series starts
    
        ret_index = (1 + returns).cumprod()
        ret_index[0] = 1
    
        # equation (1) ex ante vol estimate
        day_vol = returns.ewm(ignore_na=False,
                          adjust=True,
                          com=60,   
                          min_periods=0).std(bias=False)
        vol = day_vol * np.sqrt(261) # annualise
    
        ret_index = pd.concat([ret_index, vol], axis=1)
        ret_index.columns = [oo, oo + '_Vol']

        daily_index = pd.concat([daily_index, ret_index], axis=1)

    return daily_index

In [ ]:
def backtest_strategy(data,k = k, h = h):

    pnl = pd.DataFrame(index=data.index)
    leverage = pd.DataFrame(index=data.index)
    strategy_cumm_rtns = pd.DataFrame(index=data.index)


    daily_index = Volatility_scale(data)

    company = data.columns


    for oo in company:
        df = pd.concat([daily_index[oo], daily_index[oo+"_Vol"]], axis=1)
        df['returns'] = df[oo].pct_change(k)
    
        df['pnl'] = 0.
        df['leverage'] = 0.
        try:
            for x, v in enumerate(df['returns']):
                if x <= k:
                # skip the first look back observations
                    continue
                if df['returns'].iloc[x-h] < tolerance:
                # negative returns, sell and hold for h day, then close position
                    if vol_flag == 1:
                        df['pnl'].iloc[x] = (df[oo].iloc[x - h] / df[oo].iloc[x] - 1) * \
                                        target_vol / df[oo+"_Vol"].iloc[x - h]
                        df['leverage'].iloc[x] = target_vol / df[oo+"_Vol"].iloc[x - h]
                    else:
                        df['pnl'].iloc[x] = (df[oo].iloc[x - h] / df[oo].iloc[x] - 1)                   
                        df['leverage'].iloc[x] = 1.
                elif df['returns'].iloc[x-1] > tolerance:
                # positive returns, buy and hold for h day, then close position
                    if vol_flag == 1:
                        df['pnl'].iloc[x] = (df[oo].iloc[x] / df[oo].iloc[x - h] - 1) * \
                                        target_vol / df[oo+"_Vol"].iloc[x - h]
                        df['leverage'].iloc[x] = target_vol / df[oo+"_Vol"].iloc[x - h]
                    else:
                        df['pnl'].iloc[x] = (df[oo].iloc[x] / df[oo].iloc[x - h] - 1) 
                        df['leverage'].iloc[x] = 1.
                df = df[:-1]
        except: pass    
        # convert to cummulative index
        pnl = pd.concat([pnl, df['pnl']], axis=1)
        leverage = pd.concat([leverage, df['leverage']], axis=1)
    
        ret_index = (1 + df['pnl'][13:]).cumprod()
        ret_index[0] = 1
        strategy_cumm_rtns = pd.concat([strategy_cumm_rtns, ret_index], axis=1) 


    pnl.columns = data.columns
    leverage.columns = leverage.columns
    strategy_cumm_rtns.columns = data.columns
    df = pnl
    df['port_avg'] = df.mean(skipna = False, axis=1)

    return df['port_avg']

In [ ]:
def strategy_stat(data):
    return [data.mean(), data.std(), data.mean()/data.std(), data.min()]

In [ ]:
# Strategy = df['port_avg'].copy()
# Strategy.name = "TSMOM with Vol"
# dataport_index = (1 + df['port_avg']).cumprod()

In [ ]:
# plt.plot(backtest_strategy(US_Stock_data()))

In [ ]:
# stats = pd.DataFrame(columns=['k','h', "Mean", "Std", "Sharpe_ratio","Max drawdown"])

# for m in range(1,31):
#     for n in range(1,31):
#         temp = strategy_stat(backtest_strategy(US_Stock_data(),m,n))
#         temp.insert(0,n)
#         temp.insert(0,m)
#         stats.loc[len(stats.index)] = temp

# stats

In [ ]:
# print ("Annualized Sharpe Ratio = ", empyrical.sharpe_ratio(df['port_avg'], period='monthly'))
# print ("Annualized Mean Returns = ", empyrical.annual_return(df['port_avg'], period='monthly'))
# print ("Annualized Standard Deviations = ", empyrical.annual_volatility(df['port_avg'], period='monthly'))

In [ ]:
# print("Max Drawdown = ", empyrical.max_drawdown(df['port_avg']))
# print("Calmar ratio = ", empyrical.calmar_ratio(df['port_avg'], period='monthly'))